# Detecting Electoral Manipulation under Borda Voting Rule

This notebook contains the presentation code for the paper: Detecting Electoral Manipulation under Borda Voting Rule.

We conduct an experimental study of a linear time election manipulation detection algorithm using several syntethically generated permutations that can be seen as votes in an election. The notebook is divided in three sections:

- `Preliminaries` section installs and configures all the dependendencies required by the notebook.
- `Syntetic Data` section that deals with the experiments related to measure the accuracy of the proposed algorithm.

## Preliminaries

### Imports and general configurations

In [ ]:
import os, time, random, math
import mallows_kendall as mk
import mallows_model as mm
import numpy as np 
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd 
import permutil as pu
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import pdist, squareform, cdist

In [ ]:
sns.set(rc={'figure.figsize':(15.7,8.27)})
sns.set_style("white")

In [ ]:
from scipy.cluster.hierarchy import ClusterWarning
from warnings import simplefilter
simplefilter("ignore", ClusterWarning)

In [ ]:
clustering = AgglomerativeClustering(n_clusters=2, linkage="single")

### Auxiliary functions

In [ ]:
def compute_lower_bound(n, phi = .5):
    s = 0
    N = n*(n-1)/2
    theta = mm.phi_to_theta(phi)
    psi = np.prod(np.array([(1 - np.exp(( - n + j ) * theta))/(1 - np.exp(-theta)) for j in range(n-1)]))
    for d in range(int(N), int(math.ceil(N/2))-1, -1):
        s = s + np.exp(-theta*d)/psi*mk.num_perms_at_dist(n)[n, :int(N/2)+1][d-int(math.ceil(N/2))]
    return s

#### Hoeffding's bounds

In [ ]:
def compute_upper_bound(barS, n):
    return np.exp(-barS ** 2 / (n * (n - 1)))

#### Expected distance

In [ ]:
def barS_mallows(n, phi):
    u1, u2, u3 = 0, 0, 0

    for i in range(n):
        for j in range(n):
            if i<j:
                hij = (j - i + 1) / (1 - phi ** (j - i + 1)) - (j - i) / (1 - phi ** (j - i))

                u1 = u1 + 2 * hij - 2 * hij ** 2
                u2 = u2 + hij + (1 - hij) - 2 * (1 - hij) * hij

    barS = u2 - u1
    return barS

In [ ]:
def barS_plackett(n, w):
    u1, u2, u3 = 0, 0, 0
    
    for i in range(n):
        for j in range(n):
            if i < j:
                hij = w[i] / (w[i] + w[j])
                u1 = u1 + 2 * hij - 2 * hij ** 2
                u2 = u2 + hij + (1 - hij) - 2 * (1 - hij) * hij
    barS = u2 - u1
    return barS 

## Mallows Models

#### Generate samples using Mallows Models

In [ ]:
def sample_single_mallows(center, n=10, m=100, param=0.1):
    sample = mk.samplingMM(phi=param, m=m, n=n)
    sample = [pu.compose(perm, center) for perm in sample]
    return sample

In [ ]:
def sampler_mallows(n=10, m=100, param=0.1):
    center_1 = [i for i in range(n)]
    c1 = sample_single_mallows(center_1, n=n, m=m, param=param)
    
    center_2 = center_1[::-1]
    c2 = sample_single_mallows(center_2, n=n, m=m, param=param)
    
    c1 = [l.tolist() for l in c1]
    c2 = [l.tolist() for l in c2]
    C = c1 + c2

    return C, center_1, center_2

#### Runner

In [ ]:
def run_experiments_mallows(k, params, max_n=20, m=100, rep=100):
    Param, N = [], []
    Empirical, UpperB, LowerB = [], [], []
    
    for rep in range(rep):
        for n in range (3, max_n, 1):
            for param in params:

                C, center_1, center_2 = sampler_mallows(n=n, m=m, param=param)
                
                # Remove the winner from the whole sample
                wi = n-1
                for i in range(len(C)): 
                    rank = list(C[i])
                    rank.remove(wi)
                    C[i] = rank
                center_1.remove(wi)
                center_2.remove(wi)
                
                indexes = [i for i in range(k)] + [i for i in range(len(C) - k, len(C))]
                sampleK = [C[i] for i in indexes]
                
                
                # Count distances to apply a full clustering 
                D = pdist(sampleK, metric=mk.kendallTau) 
                D = squareform(D)
                D = pd.DataFrame(D)
                
                # Apply a full hierarchichal clustering algorithm on D
                predicted = clustering.fit_predict(D)  
                
                cluster_1 = [a for a, b in zip(sampleK, predicted) if b] 
                cluster_2 = [a for a, b in zip(sampleK, predicted) if not b]
                
                dist1 = np.mean(cdist(cluster_1, [center_1], metric=mk.kendallTau))
                dist2 = np.mean(cdist(cluster_2, [center_1], metric=mk.kendallTau))
                
                if dist1 > dist2:
                    cluster_1, cluster_2 = cluster_2, cluster_1
                    predicted = [1 if i==0 else 0 for i in predicted]
                else:
                    predicted = list(predicted)
                    
                # Loop over all elements of 1st component
                count = 0
                barS = barS_mallows(n=n, phi=param)
                for i in range(k, int(len(C)) - k):
                    
                    alpha = C[i]
                    
                    mean1 = np.mean(cdist([alpha], cluster_2, metric=mk.kendallTau))
                    mean2 = np.mean(cdist([alpha], cluster_1, metric=mk.kendallTau))
                    S = mean1 - mean2

                    if len(C) / 2 <= i:
                        S = -S
                    
                    if np.abs(barS - S) > barS:
                        count = count + 1

                Param.append(param)
                N.append(n)
                Empirical.append(count / (int(len(C)) - 2 * k))
                UpperB.append(compute_upper_bound(barS, n))
                LowerB.append(compute_lower_bound(n = n, phi=param))
                
    return  Param, N, Empirical, UpperB, LowerB

#### Experiments

In [ ]:
Phi, N, Empirical, UpperB, LowerB = run_experiments_mallows(params=[0.1, 0.3, 0.5], k=3)

#### Plotting results

In [ ]:
df1 = pd.DataFrame({"$\phi$": Phi, "n": N, "Error": Empirical, "Error type":"Empirical"})
df2 = pd.DataFrame({"$\phi$": Phi, "n": N, "Error": LowerB,    "Error type":"Lower bound"})
df3 = pd.DataFrame({"$\phi$": Phi, "n": N, "Error": UpperB,    "Error type":"Upper bound"})
df = pd.concat([df1, df2, df3], ignore_index=True)

In [ ]:
fig = plt.figure()
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
palette = sns.color_palette("gray", 3)

axs[0].set_title("Dispersion $\phi = 0.1$")
axs[1].set_title("Dispersion $\phi = 0.3$")
axs[2].set_title("Dispersion $\phi = 0.5$")

line=sns.lineplot(data=df[df["$\phi$"]==0.1], x="n", y="Error", style="Error type", hue="Error type", ax=axs[0], palette=palette)
line=sns.lineplot(data=df[df["$\phi$"]==0.3], x="n", y="Error", style="Error type", hue="Error type", ax=axs[1], palette=palette)
line=sns.lineplot(data=df[df["$\phi$"]==0.5], x="n", y="Error", style="Error type", hue="Error type", ax=axs[2], palette=palette)

plt.tight_layout()

In [ ]:
fig.savefig("img/MM.pdf", bbox_inches='tight')

## Plackett-Luce

#### Plackett-Luce sample

In [ ]:
def plackett_luce_sample(n, m, w=None):
    sample = np.zeros((m, n))
    
    if w is None: 
        w = np.array([np.exp(i) for i in reversed(range(n))])
        
    for idx in range(m):
        ordering = []
        bucket = np.arange(n, dtype=int) 
        for i in range(n):
            j = np.random.choice(bucket, p=w[bucket] / w[bucket].sum())
            ordering.append(j)
            bucket = bucket[bucket != j]
        sample[idx] = np.argsort(ordering).copy()
    return sample

In [ ]:
def generate_samples_plackett(n, m, w):
    center = [i for i in range(n)]
    c1 = plackett_luce_sample(m=m, n=n, w=w)
    c1 = [pu.compose(perm, center) for perm in c1]

    C = [l.tolist() for l in c1]
    return C, center

In [ ]:
def sampler_plackett(n=10, m=100, param=1):
    w = np.array([np.exp((n - i) / param) for i in range(n)])
    c1, center_1 = generate_samples_plackett(n=n, m=m, w=w)

    w = np.array([np.exp(i / param) for i in range(n)])
    c2, center_2 = generate_samples_plackett(n=n, m=m, w=w)
    C = [*c1, *c2]
    
    return C, center_1, center_2

#### Runner

In [ ]:
def run_experiments_plackett(k, params, max_n=20, m=100, rep=100):
    Param, N = [], []
    Empirical, UpperB, LowerB = [], [], []
    
    for rep in range(rep):
        for n in range (3, max_n, 1):
            for param in params:

                C, center_1, center_2 = sampler_plackett(n=n, m=m, param=param)
                
                # Remove the winner from the whole sample
                wi = n-1
                for i in range(len(C)): 
                    rank = list(C[i])
                    rank.remove(wi)
                    C[i] = rank
                center_1.remove(wi)
                center_2.remove(wi)
                
                indexes = [i for i in range(k)] + [i for i in range(len(C) - k, len(C))]
                sampleK = [C[i] for i in indexes]
                
                
                # Count distances to apply a full clustering 
                D = pdist(sampleK, metric=mk.kendallTau) 
                D = squareform(D)
                D = pd.DataFrame(D)
                
                # Apply a full hierarchichal clustering algorithm on D
                predicted = clustering.fit_predict(D)  
                
                cluster_1 = [a for a, b in zip(sampleK, predicted) if b] 
                cluster_2 = [a for a, b in zip(sampleK, predicted) if not b]
                
                dist1 = np.mean(cdist(cluster_1, [center_1], metric=mk.kendallTau))
                dist2 = np.mean(cdist(cluster_2, [center_1], metric=mk.kendallTau))
                
                if dist1 > dist2:
                    cluster_1, cluster_2 = cluster_2, cluster_1
                    predicted = [1 if i==0 else 0 for i in predicted]
                else:
                    predicted = list(predicted)
                    
                # Loop over all elements of 1st component
                count = 0
                barS = barS_plackett(n=n, w=np.array([np.exp(i / param) for i in range(n)]))
                for i in range(k, int(len(C)) - k):
                    
                    alpha = C[i]
                    
                    mean1 = np.mean(cdist([alpha], cluster_2, metric=mk.kendallTau))
                    mean2 = np.mean(cdist([alpha], cluster_1, metric=mk.kendallTau))
                    S = mean1 - mean2

                    if len(C) / 2 <= i:
                        S = -S
                    
                    if np.abs(barS - S) > barS:
                        count = count + 1

                Param.append(param)
                N.append(n)
                Empirical.append(count / (int(len(C)) - 2 * k))
                UpperB.append(compute_upper_bound(barS, n))
                
    return  Param, N, Empirical, UpperB

#### Experiments

In [ ]:
Z, N, Empirical, UpperB = run_experiments_plackett(params=[1, 2, 3], k=3)

#### Plotting results

In [ ]:
df1 = pd.DataFrame({"$z$": Z, "n": N, "Error": Empirical, "Error type":"Empirical"})
df2 = pd.DataFrame({"$z$": Z, "n": N, "Error": UpperB,    "Error type":"Upper Bound"})
df  = pd.concat([df1, df2], ignore_index=True)

In [ ]:
fig = plt.figure()
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
palette = sns.color_palette("gray", 2)

axs[0].set_title("Model $z = 1$")
axs[1].set_title("Model $z = 2$")
axs[2].set_title("Model $z = 3$")

line=sns.lineplot(data=df[df["$z$"]==1], x="n", y="Error", style="Error type", hue="Error type", ax=axs[0], palette=palette)
line=sns.lineplot(data=df[df["$z$"]==2], x="n", y="Error", style="Error type", hue="Error type", ax=axs[1], palette=palette)
line=sns.lineplot(data=df[df["$z$"]==3], x="n", y="Error", style="Error type", hue="Error type", ax=axs[2], palette=palette)

plt.tight_layout()

In [ ]:
fig.savefig("img/PL.pdf", bbox_inches='tight')